<a href="https://colab.research.google.com/github/xKDR/Shedding-light-on-the-Russia-Ukraine-war/blob/main/reproducible_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
using Shapefile
using DataFrames
using NighttimeLights
using Rasters
using CSV
using Dates

In [ ]:
START_YEAR = 2017  # @param {type:"integer"}

END_YEAR = 2019  # @param {type:"integer"}

In [ ]:
# Define path constants
FIGURES_DIRECTORY = "figures"
TABLES_DIRECTORY = "tables"
VECTOR_DIRECTORY = "data/vector"
TABULAR_DIRECTORY = "data/tabular"
RASTER_DIRECTORY = "data/raster"

# Check and create folders using a loop
directories = [
    FIGURES_DIRECTORY,
    TABLES_DIRECTORY,
    VECTOR_DIRECTORY,
    TABULAR_DIRECTORY,
    RASTER_DIRECTORY
]

for path in directories
    if !isdir(path)
        mkpath(path)
        println("Created folder: $path")
    end
end

In [ ]:
# @title 3.3.1 Download the annual nighttime lights data

# Define your EOG credentials
USERNAME = "ayushpatnaik@gmail.com" # @param {"type":"string"}
PASSWORD = "n3zaCXM5TtZV3Hb" # @param {"type":"string"}

CLIENT_ID = "eogdata_oidc" # @param {"type":"string"}
CLIENT_SECRET = "2677ad81-521b-4869-8480-6d05b9e57d48" # @param {"type":"string"}

function download_link(link)
    """
    Function to download image from EOG using the API

    Parameters:
    -----------
    link :: String

    Returns:
    --------
    None

    Notes:
    ------
    Downloads image in RASTER_DIRECTORY
    """
    output_file = joinpath(RASTER_DIRECTORY, basename(link))
    if isfile(output_file)
        println("File $output_file already exists. Skipping download.")
        return
    end

    params = Dict(
        "client_id" => CLIENT_ID,
        "client_secret" => CLIENT_SECRET,
        "username" => USERNAME,
        "password" => PASSWORD,
        "grant_type" => "password"
    )
    token_url = "https://eogauth.mines.edu/auth/realms/master/protocol/openid-connect/token"
    response = HTTP.post(token_url, [], params)
    access_token_dict = JSON.parse(String(response.body))
    access_token = access_token_dict["access_token"]
    data_url = link
    auth = "Bearer " * access_token
    headers = Dict("Authorization" => auth)
    response = HTTP.get(data_url, headers)

    open(output_file, "w") do f
        write(f, response.body)
    end
    println("File $output_file downloaded successfully.")
end

base_url = "https://eogdata.mines.edu/nighttime_light/annual/v21/{}/"

for year in START_YEAR:END_YEAR
    url = string("https://eogdata.mines.edu/nighttime_light/annual/v21/", year, "/VNL_v21_npp_", year, "_global_vcmslcfg_c202205302300.median_masked.dat.tif.gz")
    print(url)
    download_link(url)
end